# Applying Regexes To 10-Ks

### Introduction

In this notebook you will apply regexes to find useful financial information in 10-Ks. In particular, you will use what you learned in previous lessons to extract text from Items 1A, 7, and 7A.

# Getting Data

In this lesson, we will be working with the 2018, 10-K from Apple. In the code below, we will use the `apple_10k.txt` file, which contains the HTML data from [this 10-K](https://www.sec.gov/Archives/edgar/data/320193/000032019318000145/0000320193-18-000145.txt) directly from the SEC website. We will learn more about the `requests` library in a later lesson. We will save the HTML data into a string variable named `apple`, as shown below:

In [5]:
import re

with open('apple_10k.txt', 'r') as f:
    apple = f.read()

If we print the `raw_10k` string we will see that it has many sections. In the code below, we print part of the `raw_10k` string:

In [6]:
print(apple[0:500])

<SEC-DOCUMENT>0000320193-18-000145.txt : 20181105
<SEC-HEADER>0000320193-18-000145.hdr.sgml : 20181105
<ACCEPTANCE-DATETIME>20181105080140
ACCESSION NUMBER:		0000320193-18-000145
CONFORMED SUBMISSION TYPE:	10-K
PUBLIC DOCUMENT COUNT:		88
CONFORMED PERIOD OF REPORT:	20180929
FILED AS OF DATE:		20181105
DATE AS OF CHANGE:		20181105

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			APPLE INC
		CENTRAL INDEX KEY:			0000320193
		STANDARD INDUSTRIAL CLASSIFICATION:	ELECTRONIC COMPUTERS [3571]
		IRS


# TODO: Regexes for Tags

For our purposes, we are only interested in the sections that contain the 10-K information. All the sections, including the 10-K, are contained within the `<DOCUMENT>` and `</DOCUMENT>` tags. Each section within the document tags is clearly marked by a `<TYPE>` tag followed by the name of the section. In the code below, write three regular expressions:

1. A regex to find the `<DOCUMENT>` tag

2. A regex to find the `</DOCUMENT>` tag

3. A regex to find all the sections marked by the `<Type>` tag

In [7]:
import re

# Write regexes
doc_start_pattern = re.compile(r'<DOCUMENT>')
doc_end_pattern = re.compile(r'</DOCUMENT>')
type_pattern = re.compile(r'<TYPE>[^\n]+')

# TODO: Create Lists with Span Indices

Now, that you have the regexes, use the `.finditer()` method to match the regexes in the `raw_10k`. In the code below, create 3 lists:

1. A list that holds the `.end()` index of each match of `doc_start_pattern`

2. A list that holds the `.start()` index of each match of `doc_end_pattern`

3. A list that holds the name of section from each match of `type_pattern`

In [8]:
# Create 3 lists with the span idices for each regex
doc_start_is = [i.start()  for i in doc_start_pattern.finditer(apple)]
doc_end_is = [i.end() for i in doc_end_pattern.finditer(apple)]
doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(apple)]

# TODO: Create a Dictionary for the 10-K

In the code below, create a dictionary which has the key `10-K` and as value the contents of the `10-K` section found above. To do this, create a loop, to go through all the sections found above, and if the section type is `10-K` then save it to the dictionary. Use the indices in  `doc_start_is` and `doc_end_is`to slice the `raw_10k` file.

In [9]:
document = {}

# Create a loop to go through each section type and save only the 10-K section in the dictionary
for doc_type, doc_start_i, doc_end_i in zip(doc_types, doc_start_is, doc_end_is):
    if doc_type == '10-K':
        document[doc_type] = apple[doc_start_i:doc_end_i]


# TODO: Find Item 1A, 7, and 7A

Our task now is to use regular expression to find the items of interest. The items in this `document` can be found in four different patterns. For example Item 1A can be found in either of the following patterns:

1. `>Item 1A`

2. `>Item&#160;1A` 

3. `>Item&nbsp;1A`

4. `ITEM 1A` 

In the code below write a single regular expression that can match all four patterns for Items 1A, 7, and 7A. Then use the `.finditer()` method to match the regex to `document['10-K']`. Finally create a for loop to print the `matches`.

In [10]:
# Write the regex
regex = re.compile(
    r'(>Item(\s|&#160;|&nbsp;)(1A|7A|7)\.{0,1})|(ITEM\s(1A|7A|7))')

# Use finditer to math the regex
matches = regex.finditer(document['10-K'])

# Write a for loop to print the matches
for match in matches:
    print(match)


<_sre.SRE_Match object; span=(38328, 38337), match='>Item 1A.'>
<_sre.SRE_Match object; span=(46158, 46166), match='>Item 7.'>
<_sre.SRE_Match object; span=(47291, 47300), match='>Item 7A.'>
<_sre.SRE_Match object; span=(119141, 119150), match='>Item 1A.'>
<_sre.SRE_Match object; span=(333328, 333336), match='>Item 7.'>
<_sre.SRE_Match object; span=(729994, 730003), match='>Item 7A.'>


If your regex is written correctly, the only matches above should be those for Items 1A, 7, and 7A. You should notice also, that each item is matched twice. This is because each item appears first in the index and then in the corresponding section. We will now have to remove the matches that correspond to the index. We will do this using Pandas in the next section.

# Remove Matches that Correspond to the Index

We will remove the matches that correspond to the index using a Pandas Dataframe. We will do this in a couple of steps.

### TODO: Create a Pandas DataFrame

In the code below create a pandas dataframe with the following column names: `'item','start','end'`. In the `item` column save the `match.group()` in lower case letters, in the ` start` column save the `match.start()`, and in the `end` column save the ``match.end()`. 

In [11]:
# import pandas
import pandas as pd

# Matches
matches = regex.finditer(document['10-K'])

# Create the dataframe
test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])
test_df.columns = ['item', 'start', 'end']
test_df['item'] = test_df.item.str.lower()

# Display the dataframe
test_df


,item,start,end
0,>item 1a.,38328,38337
1,>item 7.,46158,46166
2,>item 7a.,47291,47300
3,>item 1a.,119141,119150
4,>item 7.,333328,333336
5,>item 7a.,729994,730003


### TODO: Eliminate Unnecessary Characters

As we can see, our dataframe, in particular the `item` column, contains some unnecessary characters such as `>` and periods `.`. In some cases, we will also get unicode characters such as `&#160;` and `&nbsp;`. In the code below, use the Pandas dataframe method `.replace()` with the keyword `regex=True` to replace all whitespaces, the above mentioned unicode characters, the `>` character, and the periods from our dataframe. We want to do this because we want to use the `item` column as our dataframe index later on.

In [12]:
# Get rid of unnesesary charcters from the dataframe
test_df.replace('&#160;', ' ', regex=True, inplace=True)
test_df.replace('nbsp', ' ', regex=True, inplace=True)
test_df.replace(' ', '', regex=True, inplace=True)
test_df.replace('>', ' ', regex=True, inplace=True)
test_df.replace('\.', ' ', regex=True, inplace=True)

# display the dataframe
test_df


,item,start,end
0,item1a,38328,38337
1,item7,46158,46166
2,item7a,47291,47300
3,item1a,119141,119150
4,item7,333328,333336
5,item7a,729994,730003


### TODO: Remove Duplicates

Now that we have removed all unnecessary characters form our dataframe, we can go ahead and remove the Item matches that correspond to the index. In the code below use the Pandas dataframe `.drop_duplicates()` method to only keep the last Item matches in the dataframe and drop the rest. Just as precaution make sure that the `start` column is sorted in ascending order before dropping the duplicates.

In [19]:
# Drop duplicates
pos_dat = test_df.sort_values('start', ascending=True).drop_duplicates(subset=['item'], keep='last')
# Display the dataframe
pos_dat


,item,start,end
3,item1a,119141,119150
4,item7,333328,333336
5,item7a,729994,730003


# TODO: Set Item to Index

In the code below use the Pandas dataframe `.set_index()` method to set the `item`  column as the index of our dataframe.

In [20]:
# Set item as the dataframe index
pos_dat.set_index('item',inplace=True)

# display the dataframe
pos_dat


,start,end
item,,
item1a,119141,119150
item7,333328,333336
item7a,729994,730003


# TODO: Get The Financial Information From Each Item

The above dataframe contains the starting and end index of each match for Items 1A, 7, and 7A. In the code below, save all the text from the starting index of `item1a` till the starting index of `item7` into a variable called `item_1a_raw`. Similarly,  save all the text from the starting index of `item7` till the starting index of `item7a` into a variable called `item_7_raw`. Finally,  save all the text from the starting index of `item7a` till the end of `document['10-K']` into a variable called `item_7a_raw`. You can accomplish all of this by making the correct slices of `document['10-K']`.

In [21]:
# Get Item 1a
item_1a_raw = document['10-K'][pos_dat['start'].loc['item1a']:pos_dat['start'].loc['item7']]

# Get Item 7
item_7_raw = document['10-K'][pos_dat['start'].loc['item7']:pos_dat['start'].loc['item7a']]

# Get Item 7a
item_7a_raw = document['10-K'][pos_dat['start'].loc['item7a']:]

KeyError: 'the label [item1a] is not in the [index]'

# TODO: Display Item 1a

Now that we have each item saved into a separate variable we can view them separately. For illustration purposes we will display Item 1a, but the other items will look similar.

In [22]:
# Display Item 1a
item_1a_raw

NameError: name 'item_1a_raw' is not defined

We can see that the items looks pretty messy, they contain HTML tags, Unicode characters, etc... Before we can do a proper Natural Language Processing in these items we need to clean them up. This means we need to remove all HTML Tags, unicode characters, etc... In principle we could do this using regex substitutions as we learned previously, but his can be rather difficult. Luckily, packages already exist that can do all the cleaning for us, such as **Beautifulsoup**, which will be the topic of our next lessons.

# Solution

[Solution notebook](applying_regexes_10ks_solution.ipynb)